## Load in packages and set up environment

In [1]:
using Pkg
Pkg.activate(".")

Activating environment at `~/Documents/Stanford/Stefan/EBCrossFitPaper/Project.toml`


In [2]:
using EBayes
using EBayesDatasets

In [3]:
using DataDeps
using CSV
using DataFrames
using Random
using Query

## Load dataset and preprocess

In [4]:
crime_path = @datadep_str "communities-and-crime"
crime_df = CSV.File("$crime_path/CommViolPredUnnormalizedData.txt", header=false) |>
           DataFrame
first(crime_df,6)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
,String,String,String,String,Int64,Int64,Float64,Float64
1,BerkeleyHeightstownship,NJ,39,5320,1,11980,3.1,1.37
2,Marpletownship,PA,45,47616,1,23123,2.82,0.8
3,Tigardcity,OR,?,?,1,29344,2.43,0.74
4,Gloversvillecity,NY,35,29443,1,16656,2.4,1.7
5,Bemidjicity,MN,7,5068,1,11245,2.76,0.53
6,Springfieldcity,MO,?,?,1,140494,2.45,2.51


In [5]:
crime_colnames = CSV.File(EBayesDatasets.crime_colnames_path,
                     header=false) |>
          DataFrame
names!(crime_df, Symbol.(crime_colnames[:,1]))
first(crime_colnames,6)

,Column1
,String
1,communityname
2,state
3,countyCode
4,communityCode
5,fold
6,population


In [6]:
crime_df_filt = @from i in crime_df begin
                @where i.nonViolPerPop != "?"
                @select i
                @collect DataFrame
    end

crime_df_filt = crime_df_filt |>
                @mutate(nonViolPerPop = parse.(Float64, _.nonViolPerPop)) |>
                DataFrame; 
size(crime_df_filt)

(2118, 147)

Non-violent crime rate is the quantity we will try to predict:

In [7]:
crime_rate = crime_df_filt[:,:nonViolPerPop] ./ 100_000
findmax(crime_rate), findmin(crime_rate)

((0.2711976, 1817), (0.0011679000000000001, 373))

In [8]:
mean(crime_rate)

0.04908241803588291

Let us also record the population of every community and the number of crimes.

In [9]:
npopulation = crime_df_filt[:,:population]
ncrimes = Int64.(round.(crime_df_filt[:,:nonViolPerPop] ./ 100_000 .* npopulation));
extrema(ncrimes)

(14, 451432)

In [10]:
sum(ncrimes)/sum(npopulation)

0.062310410572125594

In [11]:
ncommunities = size(crime_df_filt,1)

2118

## Set up XGBoost and EBCF models

In [12]:
using MLJ
using MLJBase

In [13]:
MLJ.@load XGBoostRegressor
xgb_tree = XGBoostRegressor(max_depth=5)
r_num_round = range(xgb_tree, :num_round, lower=2, upper=100)
r_eta = range(xgb_tree, :eta, lower=0.01, upper=1.0)
nested_ranges = [r_num_round, r_eta]
tuned_XGBoost = TunedModel(model=xgb_tree,
                           tuning=Grid(resolution=8),
                           resampling=CV(nfolds=5),
                           ranges=nested_ranges,
                           measure=rms);

In [14]:
ebcf_xgb = EBayesCrossFit(tuned_XGBoost)

EBayesCrossFit{MLJ.DeterministicTunedModel{Grid,XGBoostRegressor},Int64}(DeterministicTunedModel @ 1…61, 5)

## Subsample dataset and apply methods

In [15]:
using Distributions

In [16]:
B_subsample_200 = 200

200

In [17]:
# Hypergeometric(s, f, n)  s successes, f failures, n trials
# (s k)(f n-k)/(s+f n)

# prob k incidents =  (from violent results choose k *  from non violent results choose n-k) /

# so here; Hypergeometric(Violent, Non-Violent, B_subsample)

Random.seed!(1)
subsampled_crimes_200 = rand.( Hypergeometric.(ncrimes, npopulation .- ncrimes, B_subsample_200))
subsampled_crime_rate_200 = subsampled_crimes_200 ./ B_subsample_200;

In [18]:
vst_crimes_200 =  NormalSamples( sqrt.(subsampled_crime_rate_200),
                             fill( sqrt(1/B_subsample_200/4), ncommunities));

In [19]:
# sanity check: Check approximate variance stabilization
var( vst_crimes_200.Z .- sqrt.(crime_rate ))*4*B_subsample_200

1.0485465098587923

In [20]:
mean( (subsampled_crime_rate_200 .- crime_rate).^2 ) * 1_000_000

223.87648677616622

## Apply methods that do not use covariates

In [21]:
unbiased_preds_200 = vst_crimes_200.Z #unbiased at sqrt_scale
unbiased_errors_200 = (unbiased_preds_200.^2 .- crime_rate).^2 .*1_000_000 # so that MSE is per 1_k population;
mean(unbiased_errors_200) # so that MSE is per 1_k population

223.8764867761662

In [22]:
mean(unbiased_preds_200.^2), mean(crime_rate)

(0.04879603399433428, 0.04908241803588291)

In [23]:
sure_fit_200 = fit(Normal(), SURE(), GrandMeanLocation(), vst_crimes_200)
sure_preds_200 = predict(sure_fit_200);

In [24]:
sure_errors_200 = (sure_preds_200.^2 .- crime_rate).^2 .*1_000_000 # so that MSE is per 1_k population;
mean(sure_errors_200)

184.19454193213878

## Apply covariate-powered methods

In [25]:
X = crime_df_filt[:,6:129]; #lasnames(X)
X_sub =X[:, findall( typeof.(eachcol(X)) .== Vector{Float64})]
names(X_sub)

74-element Array{Symbol,1}:
 :householdsize        
 :racepctblack         
 :racePctWhite         
 :racePctAsian         
 :racePctHisp          
 :agePct12t21          
 :agePct12t29          
 :agePct16t24          
 :agePct65up           
 :pctUrban             
 :pctWWage             
 :pctWFarmSelf         
 :pctWInvInc           
 ⋮                     
 :MedRentPctHousInc    
 :MedOwnCostPctInc     
 :MedOwnCostPctIncNoMtg
 :PctForeignBorn       
 :PctBornSameState     
 :PctSameHouse85       
 :PctSameCity85        
 :PctSameState85       
 :LandArea             
 :PopDens              
 :PctUsePubTrans       
 :LemasPctOfficDrugUn  

In [26]:
size(X_sub)

(2118, 74)

In [27]:
# sanity check 1
mean(X_sub[:, :PctUsePubTrans]) #~3.041

3.0503305004721435

In [28]:
# sanity check 2
mean(X_sub[:, :LemasPctOfficDrugUn]) # 0.98

0.9746270066100096

In [29]:
Random.seed!(1)
ebcf_fit_200 = fit(ebcf_xgb, X_sub, vst_crimes_200, verbosity=0);

In [30]:
xgboost_preds_200 = ebcf_fit_200.reg_preds
xgboost_errors_200 = (xgboost_preds_200.^2 .- crime_rate).^2 * 1_000_000
mean(xgboost_errors_200)

398.01651704741624

In [31]:
ebcf_preds_200 = predict(ebcf_fit_200)
ebcf_errors_200 = (ebcf_preds_200.^2 .- crime_rate).^2 * 1_000_000;
mean(ebcf_errors_200)

151.95378730041037

In [32]:
DataFrames.DataFrame(
             [(method="Unbiased", mse = mean(unbiased_errors_200),
                                  mse_2se = 2*std(unbiased_errors_200)/sqrt(ncommunities)),
              (method="XGBoost", mse = mean(xgboost_errors_200),
                                 mse_2se = 2*std(xgboost_errors_200)/sqrt(ncommunities)),
              (method="SURE", mse = mean(sure_errors_200),
                              mse_2se = 2*std(sure_errors_200)/sqrt(ncommunities)),
              (method="EBCF", mse = mean(ebcf_errors_200), 
                              mse_2se = 2*std(ebcf_errors_200)/sqrt(ncommunities))
         ])

,method,mse,mse_2se
,String,Float64,Float64
1,Unbiased,223.876,16.834
2,XGBoost,398.017,81.8425
3,SURE,184.195,18.9228
4,EBCF,151.954,22.2334


In [33]:
## sanity check
A_200 = ebcf_fit_200.eb_fits[1].fitted_obj
var_200 = 1/4/200
A_200/(var_200 + A_200)

0.5952464346905292

### Not reported in paper but let us also check MSEs in original scale.

In [34]:
unbiased_errors_sqrt_200 = (unbiased_preds_200 .- sqrt.(crime_rate)).^2 .*1_000 # so that MSE is per 1_k population;
xgboost_errors_sqrt_200 = (xgboost_preds_200 .- sqrt.(crime_rate)).^2 .*1_000 # so that MSE is per 1_k population;
sure_errors_sqrt_200 = (sure_preds_200 .- sqrt.(crime_rate)).^2 .*1_000 # so that MSE is per 1_k population;
ebcf_errors_sqrt_200 = (ebcf_preds_200 .- sqrt.(crime_rate)).^2 .*1_000 # so that MSE is per 1_k population;

DataFrames.DataFrame(
             [(method="Unbiased", mse = mean(unbiased_errors_sqrt_200),
                                  mse_2se = 2*std(unbiased_errors_sqrt_200)/sqrt(ncommunities)),
              (method="XGBoost", mse = mean(xgboost_errors_sqrt_200),
                                 mse_2se = 2*std(xgboost_errors_sqrt_200)/sqrt(ncommunities)),
              (method="SURE", mse = mean(sure_errors_sqrt_200),
                                 mse_2se = 2*std(sure_errors_sqrt_200)/sqrt(ncommunities)),
              (method="EBCF", mse = mean(ebcf_errors_sqrt_200), mse_2se = 2*std(ebcf_errors_sqrt_200)/sqrt(ncommunities))
         ])

,method,mse,mse_2se
,String,Float64,Float64
1,Unbiased,1.32748,0.0980968
2,XGBoost,1.72889,0.220073
3,SURE,0.938034,0.0605537
4,EBCF,0.734621,0.054488


# Repeat analysis above with B=500 subsampling

In [35]:
B_subsample_500 = 500
Random.seed!(1)
subsampled_crimes_500 = rand.( Hypergeometric.(ncrimes, npopulation .- ncrimes, B_subsample_500))
subsampled_crime_rate_500 = subsampled_crimes_500 ./ B_subsample_500;

In [36]:
vst_crimes_500 =  NormalSamples( sqrt.(subsampled_crime_rate_500),
                             fill( sqrt(1/B_subsample_500/4), length(subsampled_crime_rate_500)));
var( vst_crimes_500.Z .- sqrt.(crime_rate ))*4*B_subsample_500

0.9977559100448699

In [37]:
unbiased_preds_500 = vst_crimes_500.Z #unbiased at sqrt_scale
unbiased_errors_500 = (unbiased_preds_500.^2 .- crime_rate).^2 .*1_000_000 # so that MSE is per 1_k population;
@show mean(unbiased_errors_500)

sure_fit_500 = fit(Normal(), SURE(), GrandMeanLocation(), vst_crimes_500)
sure_preds_500 = predict(sure_fit_500);

sure_errors_500 = (sure_preds_500.^2 .- crime_rate).^2 .*1_000_000 # so that MSE is per 1_k population;
@show mean(sure_errors_500);

mean(unbiased_errors_500) = 92.23322558636448
mean(sure_errors_500) = 85.58825495773978


In [38]:
Random.seed!(1)
ebcf_fit_500 = fit(ebcf_xgb, X_sub, vst_crimes_500, verbosity=0);

In [39]:
xgboost_preds_500 = ebcf_fit_500.reg_preds
xgboost_errors_500 = (xgboost_preds_500.^2 .- crime_rate).^2 * 1_000_000
@show mean(xgboost_errors_500)

ebcf_preds_500 = predict(ebcf_fit_500)
ebcf_errors_500 = (ebcf_preds_500.^2 .- crime_rate).^2 * 1_000_000;
@show mean(ebcf_errors_500)

mean(xgboost_errors_500) = 370.1592472206465
mean(ebcf_errors_500) = 78.50251399181964


78.50251399181964

In [40]:
DataFrames.DataFrame(
             [(method="Unbiased", mse = mean(unbiased_errors_500),
                                  mse_2se = 2*std(unbiased_errors_500)/sqrt(ncommunities)),
              (method="XGBoost", mse = mean(xgboost_errors_500),
                                 mse_2se = 2*std(xgboost_errors_500)/sqrt(ncommunities)),
              (method="SURE", mse = mean(sure_errors_500),
                                 mse_2se = 2*std(sure_errors_500)/sqrt(ncommunities)),
              (method="EBCF", mse = mean(ebcf_errors_500), mse_2se = 2*std(ebcf_errors_500)/sqrt(ncommunities))
         ])

,method,mse,mse_2se
,String,Float64,Float64
1,Unbiased,92.2332,7.09465
2,XGBoost,370.159,78.5524
3,SURE,85.5883,7.16978
4,EBCF,78.5025,10.2882


In [41]:
## sanity check
A_500 = ebcf_fit_500.eb_fits[1].fitted_obj
var_500 = 1/4/500
A_500/(var_500 + A_500)

0.7985097313143861

### Not reported in paper but let us also check MSEs in original scale.

In [42]:
unbiased_errors_sqrt_500 = (unbiased_preds_500 .- sqrt.(crime_rate)).^2 .*1_000 # so that MSE is per 1_k population;
xgboost_errors_sqrt_500 = (xgboost_preds_500 .- sqrt.(crime_rate)).^2 .*1_000 # so that MSE is per 1_k population;
sure_errors_sqrt_500 = (sure_preds_500 .- sqrt.(crime_rate)).^2 .*1_000 # so that MSE is per 1_k population;
ebcf_errors_sqrt_500 = (ebcf_preds_500 .- sqrt.(crime_rate)).^2 .*1_000 # so that MSE is per 1_k population;

DataFrames.DataFrame(
             [(method="Unbiased", mse = mean(unbiased_errors_sqrt_500),
                                  mse_2se = 2*std(unbiased_errors_sqrt_500)/sqrt(ncommunities)),
              (method="XGBoost", mse = mean(xgboost_errors_sqrt_500),
                                 mse_2se = 2*std(xgboost_errors_sqrt_500)/sqrt(ncommunities)),
              (method="SURE", mse = mean(sure_errors_sqrt_500),
                                 mse_2se = 2*std(sure_errors_sqrt_500)/sqrt(ncommunities)),
              (method="EBCF", mse = mean(ebcf_errors_sqrt_500), mse_2se = 2*std(ebcf_errors_sqrt_500)/sqrt(ncommunities))
         ])

,method,mse,mse_2se
,String,Float64,Float64
1,Unbiased,0.499599,0.0324802
2,XGBoost,1.60436,0.208687
3,SURE,0.427787,0.0263583
4,EBCF,0.36962,0.0248903
